In [2]:
##
## ---------------------------------------------------------------------------
##

# Import SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, FloatType, StructType, StructField, IntegerType,  StringType

## the distance function
from scipy.spatial import distance
## creates the feature vector
from pyspark.ml.feature import VectorAssembler
## import numpy
import numpy as np

from pyspark.sql.functions import max, abs

from pyspark.sql.functions import greatest

import time

import pyspark.sql.functions as F

from statistics import mode

import matplotlib.pyplot as plt

from functools import reduce

from pyspark.ml.linalg import Vectors

import numpy as np

import heapq


## start session
spark = SparkSession.builder.appName("SparkLAESAKnn").getOrCreate()




##DEFINES A UDF FROM L2DIST THE DATAFRAME FROM A QUERY SET
distance_udf = F.udf(lambda x,y: float(distance.euclidean(x, y)), DoubleType())

def simpleF(oq):
    return F.udf(lambda x: float(distance.euclidean(x, oq)), DoubleType())


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/25 09:34:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
##
## OBJECT QUERY
##

## defines query object and 
oq = [np.random.rand(1)[0],np.random.rand(1)[0]]
   
##
##
## 

##
## NEIGHBORS
##
    
## defines neighbrs amount
k = 10



##
##
##


##
## PIVOTS
##

# define pivots amount
pivots = 2


##
##
##


##
## DF
##

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("coord_x", FloatType(), True),
    StructField("coord_y", FloatType(), True)])

df = spark.read.csv("/home/weber/Documents/coordDF100.csv",schema=schema)        
size = df.count()
##SMALL SANITY CHECK - @PRODUCTION TESTAR LOADING
df = df.na.drop()

## define the struct for the dimensional feature vector
cNames = df.columns
cNames.remove("id")
assembler = VectorAssembler(
    inputCols=cNames,
    outputCol="fv")

## appends the fv into the dataframe as column
df = assembler.transform(df)


# creates random pivots
pivots_list = [(i+1,[np.random.rand(1)[0],np.random.rand(1)[0]]) for i in range(pivots)]

## calculates the distance from oi to pivots
for i in range(len(pivots_list)):
    df = df.withColumn(f'distances_oi_pivot{i+1}', simpleF(pivots_list[i][1])(F.col('fv')))

    
##
##
##
lista_columns = []
for i in range(len(pivots_list)):
    df = df.withColumn(f"|d(oq,p{i+1})-d(oi,p{i+1})|", \
        abs (distance.euclidean(oq, pivots_list[i][1]) - \
        F.col(f'distances_oi_pivot{i+1}')))

    lista_columns.append(f"|d(oq,p{i+1})-d(oi,p{i+1})|")

df = df.withColumn("lower_bound", greatest(*[col_name for col_name in lista_columns]))
df = df.orderBy(df["lower_bound"])


df.show()




+---+-------+-------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+
| id|coord_x|coord_y|                  fv|distances_oi_pivot1|distances_oi_pivot2| |d(oq,p1)-d(oi,p1)|| |d(oq,p2)-d(oi,p2)||         lower_bound|
+---+-------+-------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+
| 34|0.98443|0.59886|[0.98443001508712...|0.37026331517511335| 0.5393689557183697| 0.03872940134059161| 0.03731194753112688| 0.03872940134059161|
| 53|0.56908|0.31323|[0.56907999515533...|0.36699986378650096| 0.4730217073046517|   0.041992852729204|0.029035300882591153|   0.041992852729204|
| 45|0.52276|0.23348|[0.52275997400283...|0.45392399103067677| 0.5454484108921628| 0.04493127451497181|0.043391402704919924| 0.04493127451497181|
|  8|0.37089|0.29837|[0.37088999152183...| 0.4539840145156477| 0.4897422435498806|0.044991297999942725|0.012314764637362252|

In [10]:
def add_to_queue(row, oq, k, pq, r_laesa):    
    h += 1
    if (len(pq) < k):
        heapq.heappush(pq, (-(distance.euclidean(oq, row.fv)),row.id))
        print(pq)
#    else:
#        nextDist = -(distance.euclidean(oq, row.fv))
#        if (nextDist > pq[0][0]): 
#            heapq.heappush(pq, (nextDist, row.id))
#            heapq.heappop(pq) 
#            r_laesa = pq[0][0]
#    if (k < h < df.count() and row.lower_bound >= -(r_laesa)):
#        print("I had had to be gone")
        
def laesa_queue(df, oq, k):
    pq = []
    r_laesa = float('inf')
    h=0
    df.foreach(lambda row: add_to_queue(row, oq, k, pq, r_laesa))
    drops = df.count()-h
    return(pq, drops)


laesa_queue(df, oq, k)

23/03/25 09:41:30 ERROR Executor: Exception in task 0.0 in stage 33.0 (TID 136)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 540, in func
    return f(iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 1160, in processPartition
    f(x)
  File "/opt/spark/python/pyspark/util.py", line 81, in wrapper
    return f(*args, **k

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 136) (localhost executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rd

In [16]:
# Create an empty list
pq = []

# Convert the list to a priority queue
heapq.heapify(pq)

heapq.heappush(pq, ("texto"))

print(pq)

['texto']


In [51]:
print(len(pq))

10
